In [1]:
import os
import sys
import pandas as pd

nb_path = os.getcwd()
proj_path = nb_path.split('Notebooks')[0]
tools_path = proj_path + "/Analysis_Tools"
sys.path.insert(0, tools_path)
from Parsers import Game_Predictor
from Setup import Directory_setup
from importlib import reload
reload(Game_Predictor)
reload(Directory_setup)

setup = Directory_setup.Create_Directories()
project_path = setup.project_path

# 2020 NFL Weekly Game Predictions
#### This Notebook makes spread picks based on the Expected Game Outcome (EGO) as calculated by our DVOA Model trained on Data from the 2006-2019 NFL seasons.
<br>
<br>
Below is the Data and Predictions for this week:

In [2]:
week = 8
season = 2020
Data = Game_Predictor.NFL_Game_Predictor(project_path, week, season, updateType='Week')
Spread_Target_DF = Data.Spread_Targets
print(f'\nWeek {week} Evaluation:\n')
print(Spread_Target_DF)

Analyzing Week: 8 Games ...


NameError: name 'Prediction_Helper' is not defined

## How The Model is Built

The Model is trained with Data on every game from 2006 to 2019 by looking at the spread, the score and the Teams' WDVOA Difference for each game week 6 and later. WDVOA is a measure of a teams' overall strength that takes into account all facets of the game and is weighted toward the more recent performances. This stat is published every week by football outsiders. The premmise of the model is as follows:
1. Log the score and Teams' WDVOA difference for every game.
2. Create a "Map" of average scoring margin and WDVOA Difference such that for any game with WDVOA Difference X we can find an expected score Y. This is our Expected Game Outcome (EGO).
3. Divide this map by Home and Away Teams. Teams playing at home have an advantage that is not considered in the WDVOA stat so we need to seperate the predictions according to whether the teams are home and away.
<br>
<br>
The resultant Map/Model can be viewed below:

In [ ]:
map = pd.read_csv('Models/All Seasons Scores Grouped By WDVOA Diff.csv')
map #Delete "#" at beginning of line to show output

## Evaluating the Model
With this model we can make a pick for every game in our 2006-2019 dataset by comparing our EGO to the spread and determing the Pick accordingly. The assigned Pick was then assessed relative to the actual outcome of the game to determine if right or wrong. 
<br>
<br>
The results of this evaluation are shown in the demonstation below:

In [ ]:
#Get the Data Path
DVOA_Type = 'WDVOA'
Total_Prediction_Data_Path = f'{proj_path}/Data/Total Data/ALL {DVOA_Type} Model Data.csv'
#Read the Data
dataset = pd.read_csv(Total_Prediction_Data_Path)
#Defien the columns needed for this evaluation
evaluation_df = dataset[['DVOA Pick Right', 'DVOA EGO to Spread Diff']].sort_values(by='DVOA EGO to Spread Diff', ascending=False)


In [ ]:
#Initialize Dataframe and Lists to Store REsults
results_df = pd.DataFrame()
accuracy_data = []
avg_ego_data = []
legend_data = []

#Get Betting accuracy by EGO to Spread Difference for each moving average window
moving_windows = [250, 500]
for moving_avg in moving_windows:
    legend = f'{DVOA_Type} Based Betting Accuracy - {moving_avg} points MA'
    ego_data = list(evaluation_df['DVOA EGO to Spread Diff'])
    prediction_data = list(evaluation_df['DVOA Pick Right'])
    counter=0
    for dp in range(0,len(ego_data)-moving_avg):
        avg_egoDiff = sum(ego_data[dp:(dp+moving_avg)])/moving_avg
        avg_acc = ((sum(prediction_data[dp:(dp+moving_avg)])/2)+(moving_avg/2))/moving_avg
        avg_ego_data.append(avg_egoDiff)
        accuracy_data.append(avg_acc)
        counter+=1
    legend_data += [legend for i in range(counter)]
    
#Store results into final dataframe
results_df['Betting Accuracy'] = accuracy_data
results_df['EGO to Spread Diff'] = avg_ego_data
results_df['Analysis Name'] = legend_data

In [ ]:
import plotly.express as px
fig = (px.scatter(results_df, x="EGO to Spread Diff", y="Betting Accuracy", color="Analysis Name").update_traces(mode='lines+markers', line=dict(width=1), marker=dict(size=3),))
fig.update_layout(dict(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)'))
fig.show(renderer='notebook')

For Games were our expected score and the spread differ by 1.5 to 3.7 points, we see a 55-59% success rate in our picks against the spread. For Games where our EGO are within 1.5 points of the spread, we sit at a 48 to 50% acuracy while games where our EGO is greater than 3.7 off from the spread, our accuracy declines as this difference grows. <br>
<br>
We atribute these results to indicate that when the Spread and EGO are matching (within 1.5 points), the spread is appropriate and thus there is an even 50% chance of covering the spread. Contrastingly, when our EGO differs by the spread by too much, our accuracy declines because there is something our model has failed to take into account, like an injury, so we see a decline in Accuracy in this case. The sweet spot occurs when our EGO differs by the spread enough that we have gained an advantage not factored in by the spread but not due to much that we it's attributed to unaccounted events. In this range we see a sustainable betting acuracy
<br>
<br>
Thus, when we pick games, we will look for opportunites where the EGO differs from the spread by between 1.5 to 3.7 points to maximize our chances of being in this sweet spot. 